In [ ]:
#syntax = "proto3";
#
#package tensorflow;
#option cc_enable_arenas = true;
#option java_outer_classname = "StepStatsProtos";
#option java_multiple_files = true;
#option java_package = "org.tensorflow.framework";
#option go_package = "github.com/tensorflow/tensorflow/tensorflow/go/core/framework";
#import "tensorflow/core/framework/allocation_description.proto";
#import "tensorflow/core/framework/tensor_description.proto";
#
#// An allocation/de-allocation operation performed by the allocator.
#message AllocationRecord {
#  // The timestamp of the operation.
#  int64 alloc_micros = 1;
#  // Number of bytes allocated, or de-allocated if negative.
#  int64 alloc_bytes = 2;
#}
#
#message AllocatorMemoryUsed {
#  string allocator_name = 1;
#  // These are per-node allocator memory stats.
#  int64 total_bytes = 2;
#  int64 peak_bytes = 3;
#  // The bytes that are not deallocated.
#  int64 live_bytes = 4;
#  // The allocation and deallocation timeline.
#  repeated AllocationRecord allocation_records = 6;
#
#  // These are snapshots of the overall allocator memory stats.
#  // The number of live bytes currently allocated by the allocator.
#  int64 allocator_bytes_in_use = 5;
#}
#
#// Output sizes recorded for a single execution of a graph node.
#message NodeOutput {
#  int32 slot = 1;
#  TensorDescription tensor_description = 3;
#};
#
#// For memory tracking.
#message MemoryStats {
#  int64 temp_memory_size = 1;
#  int64 persistent_memory_size = 3;
#  repeated int64 persistent_tensor_alloc_ids = 5;
#
#  int64 device_temp_memory_size = 2 [deprecated = true];
#  int64 device_persistent_memory_size = 4 [deprecated = true];
#  repeated int64 device_persistent_tensor_alloc_ids = 6 [deprecated = true];
#}
#
#// Time/size stats recorded for a single execution of a graph node.
#message NodeExecStats {
#  // TODO(tucker): Use some more compact form of node identity than
#  // the full string name.  Either all processes should agree on a
#  // global id (cost_id?) for each node, or we should use a hash of
#  // the name.
#  string node_name = 1;
#  int64 all_start_micros = 2;
#  int64 op_start_rel_micros = 3;
#  int64 op_end_rel_micros = 4;
#  int64 all_end_rel_micros = 5;
#  repeated AllocatorMemoryUsed memory = 6;
#  repeated NodeOutput output = 7;
#  string timeline_label = 8;
#  int64 scheduled_micros = 9;
#  uint32 thread_id = 10;
#  repeated AllocationDescription referenced_tensor = 11;
#  MemoryStats memory_stats = 12;
#  int64 all_start_nanos = 13;
#  int64 op_start_rel_nanos = 14;
#  int64 op_end_rel_nanos = 15;
#  int64 all_end_rel_nanos = 16;
#  int64 scheduled_nanos = 17;
#};
#
#message DeviceStepStats {
#  string device = 1;
#  repeated NodeExecStats node_stats = 2;
#}
#
#message StepStats {
#  repeated DeviceStepStats dev_stats = 1;
#};

Views:
    * graph view
    * scope view
    * op view
    * code view
Output format:
    * timeline json
    * stdout
    * pprof file
    * txt file

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.examples.tutorials.mnist import input_data


from tensorflow.python.profiler import model_analyzer
from tensorflow.python.profiler import option_builder

In [3]:
class FLAGS:
  def __init__(self):
    self.hidden1_dim =128     #hidden1 layer 的维度
    self.hidden2_dim = 32     #hidden2 layer 的维度
    self.learning_rate = 1e-2
    self.batch_size = 100
    self.max_step = 100        # 最大的训练步骤
    self.stats_per_steps = 10  # 每隔10步搜集一次RunMetadata。 可以根据需要修改
    self.data_set_dir =  '' #替换为你的本地mnist数据集路径，如果你已经下载过


TRAINING_FLAGS = FLAGS()

In [4]:
data_sets = input_data.read_data_sets(train_dir=TRAINING_FLAGS.data_set_dir,fake_data=False)
images_placeholder = tf.placeholder(tf.float32, shape=(TRAINING_FLAGS.batch_size,
                                                       mnist.IMAGE_PIXELS))
labels_placeholder = tf.placeholder(tf.int32, shape=(TRAINING_FLAGS.batch_size))

logits = mnist.inference(images_placeholder,
                         TRAINING_FLAGS.hidden1_dim,
                         TRAINING_FLAGS.hidden2_dim)
loss = mnist.loss(logits, labels_placeholder)
train_op = mnist.training(loss, TRAINING_FLAGS.learning_rate)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
mnist_profiler = model_analyzer.Profiler(graph=sess.graph)

In [6]:
run_options = tf.RunOptions(trace_level = tf.RunOptions.FULL_TRACE) #hardware + software

In [7]:
run_metadata = tf.RunMetadata()

In [8]:
feed_dict = dict()
for step in range(TRAINING_FLAGS.max_step):
    images_feed, labels_feed = data_sets.train.next_batch(TRAINING_FLAGS.batch_size, fake_data=False)
    feed_dict = {
                 images_placeholder: images_feed,
                 labels_placeholder: labels_feed,
                }    
    if step % TRAINING_FLAGS.stats_per_steps == 0:
        _, loss_value = sess.run(fetches=[train_op, loss],feed_dict=feed_dict, options=run_options, run_metadata=run_metadata)

        #将本步搜集的统计数据添加到tfprofiler实例中     
        mnist_profiler.add_step(step=step, run_meta=run_metadata)
    else:
        _, loss_value = sess.run(fetches=[train_op, loss],
                               feed_dict=feed_dict)

In [27]:
_options = {'max_depth': 100,
            'min_bytes': 0,
            'min_micros': 0,
            'min_params': 0,
            'min_float_ops': 0,
            'min_occurrence': 0,
            'order_by': 'name',
            'account_type_regexes': ['.*'],
            'start_name_regexes': ['.*'],
            'trim_name_regexes': [],
            'show_name_regexes': ['.*'],
            'hide_name_regexes': [],
            'account_displayed_op_only': False,
            'select': ['micros'],
            'step': -1,#average of all
            'output': 'stdout'}

In [25]:
#3 options dicts
#trainable_variables_parameter()
#float_operations()
#time_and_memory()

profile_graph_opts_builder = option_builder.ProfileOptionBuilder()
#option_builder.ProfileOptionBuilder.trainable_variables_parameter())

#output method1
#profile_graph_opts_builder.with_timeline_output(timeline_file='/tmp/mnist_profiler.json')
profile_graph_opts_builder.with_file_output(outfile='abc.txt')

#display options
#profile_graph_opts_builder.with_accounted_type_regexes('')# ‘.MatMul.‘, ‘.*Conv2D’, ‘.*gpu:0’
#profile_graph_opts_builder.with_node_names(show_name_regexes=['*'])

#setters start with 'with'
profile_graph_opts_builder.with_step(70)
profile_graph_opts_builder.with_max_depth(20)
profile_graph_opts_builder.account_displayed_op_only(True)

#query
profile_graph_opts_builder.select(['micros', 'occurrence'])
profile_graph_opts_builder.order_by('micros')


#view type1
mnist_profiler.profile_graph(profile_graph_opts_builder.build())

##view type2
#mnist_profiler.profile_name_scope(profile_graph_opts_builder.build())
#
##view type3
#mnist_profiler.profile_operations(profile_graph_opts_builder.build())
#
##view type4
#mnist_profiler.profile_python(profile_graph_opts_builder.build())

#advise
mnist_profiler.advise(options=model_analyzer.ALL_ADVICE)

checkers {
  key: "AcceleratorUtilizationChecker"
  value {
  }
}
checkers {
  key: "ExpensiveOperationChecker"
  value {
    reports: "top 1 operation type: MatMul, cpu: 604us, accelerator: 0us, total: 604us (50.80%)\ntop 2 operation type: ApplyGradientDescent, cpu: 153us, accelerator: 0us, total: 153us (12.87%)\ntop 3 operation type: SparseSoftmaxCrossEntropyWithLogits, cpu: 146us, accelerator: 0us, total: 146us (12.28%)"
    reports: "top 1 graph node: hidden1, cpu: 0us, accelerator: 0us, total: 0us\ntop 2 graph node: gradients, cpu: 0us, accelerator: 0us, total: 0us\ntop 3 graph node: GradientDescent, cpu: 43us, accelerator: 0us, total: 43us"
    reports: ""
  }
}
checkers {
  key: "OperationChecker"
  value {
  }
}